In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os
import cv2
from tqdm import tqdm
import random
import shutil

from face_detector import FaceDetector

In [ ]:
IMAGES_DIR = '/home/gpu2/hdd/dan/FDDB/originalPics/'
ANNOTATIONS_PATH = '/home/gpu2/hdd/dan/FDDB/FDDB-folds/'
RESULT_DIR = 'result/'
MODEL_PATH = 'model.pb'

# Collect annotated images

In [ ]:
annotations = [s for s in os.listdir(ANNOTATIONS_PATH) if s.endswith('ellipseList.txt')]
image_lists = [s for s in os.listdir(ANNOTATIONS_PATH) if not s.endswith('ellipseList.txt')]
annotations = sorted(annotations)
image_lists = sorted(image_lists)

In [ ]:
images_to_use = []
for n in image_lists:
    with open(os.path.join(ANNOTATIONS_PATH, n)) as f:
        images_to_use.extend(f.readlines())

In [ ]:
images_to_use = [s.strip() for s in images_to_use]
with open(os.path.join(RESULT_DIR, 'faceList.txt'), 'w') as f:
    for p in images_to_use:
        f.write(p + '\n')

In [ ]:
ellipses = []
for n in annotations:
    with open(os.path.join(ANNOTATIONS_PATH, n)) as f:
        ellipses.extend(f.readlines())

In [ ]:
i = 0
with open(os.path.join(RESULT_DIR, 'ellipseList.txt'), 'w') as f:
    for p in ellipses:
        
        # check image order
        if 'big/img' in p:
            assert images_to_use[i] in p
            i += 1

        f.write(p)

# Predict using trained detector

In [ ]:
face_detector = FaceDetector(MODEL_PATH, gpu_memory_fraction=0.25, visible_device_list='0')

In [ ]:
predictions = []
for n in tqdm(images_to_use):
    image_array = cv2.imread(os.path.join(IMAGES_DIR, n) + '.jpg')
    image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
    # threshold is important to set low
    boxes, scores = face_detector(image_array, score_threshold=0.05)
    predictions.append((n, boxes, scores))

In [ ]:
with open(os.path.join(RESULT_DIR, 'detections.txt'), 'w') as f:
    for n, boxes, scores in predictions:
        f.write(n + '\n')
        f.write(str(len(boxes)) + '\n')
        for b, s in zip(boxes, scores):
            ymin, xmin, ymax, xmax = b
            h, w = int(ymax - ymin), int(xmax - xmin)
            f.write('{0} {1} {2} {3} {4:.4f}\n'.format(int(xmin), int(ymin), w, h, s))

# Copy images

In [ ]:
for n in tqdm(images_to_use):
    p = os.path.join(RESULT_DIR, 'images', n + '.jpg')
    os.makedirs(os.path.dirname(p), exist_ok=True)
    shutil.copy(os.path.join(IMAGES_DIR, n) + '.jpg', p)